# Converting MICrONS to NWB

This notebook converts the 2p data from [the MICrONS dataset](https://www.microns-explorer.org/) to NWB.

## Setup
The notebook needs to be run on https://codebook.datajoint.io/ using the "IARPA MICrONS Program" Sever Option


## Progress
- [x] Eye tracking
    - [x] Minor and major radius
    - [x] Eye position (x, y)
- [x] Treadmill velocity
- [x] Trials
    - [x] Start and stop times
    - [x] Stimulus type
    - [x] Condition hash
- [x] ROI masks
- [x] Fluorescence traces
- [x] Summary images
- [x] Microscope metadata
- [ ] Raw 2p data
- [ ] Stimulus movie (need more RAM)
- [ ] Mapping to EM data
- [ ] Subject and session metadata
    - [ ] datetime of session
    - [ ] age, sex, and ID of mouse

In [ ]:
!pip install neuroconv[ophys]

In [ ]:
from datetime import datetime
from dateutil import tz

import datajoint as dj
from phase3 import nda, func, utils

from neuroconv.tools.nwb_helpers import (
    get_module,
    make_or_load_nwbfile,
    get_default_nwbfile_metadata,
)
from neuroconv.tools.roiextractors import (
    add_image_segmentation,
    add_imaging_plane,
)

from pynwb.file import Subject
from pynwb.ophys import RoiResponseSeries, Fluorescence
from pynwb.image import OpticalSeries, GrayscaleImage
from hdmf.backends.hdf5.h5tools import H5DataIO
from pynwb.behavior import PupilTracking, EyeTracking, SpatialSeries
from pynwb.base import TimeSeries, Images
import numpy as np
from datetime import datetime

def get_default_subject():
    return Subject(subject_id="001", species="Mus musculus", age="P75D/P81D", sex="M")

def get_ophys_metadata(scan_key, metadata: dict):
    device_metadata = dict(
        name="Microscope",
        description="two-photon random access mesoscope",
    )
    optical_channel_metadata = dict(
        name="OpticalChannel",
        emission_lambda=500.,
        description="An optical channel.",
    )

    all_field_data = (nda.Field & scan_key).fetch(as_dict=True)

    imaging_planes = []
    for field_data in all_field_data:
        imaging_plane_metadata = dict(
            name=f"ImagingPlane{field_data['field']}",
            description="The plane or volume being imaged by the microscope.",
            excitation_lambda=920.,
            indicator="GCaMP6",
            location="unknown",
            device=device_metadata["name"],
            optical_channel=[optical_channel_metadata],
            grid_spacing=[field_data["um_width"]/field_data["px_width"]*1e-6, field_data["um_height"]/field_data["px_height"]] ,
            grid_spacing_unit="meters",
            origin_coords=[field_data["field_x"], field_data["field_y"], field_data["field_z"]],
            origin_coords_unit="meters",
        )
        imaging_planes.append(imaging_plane_metadata)

    metadata.update(Ophys=dict(Device=[device_metadata], ImagingPlane=imaging_planes,))

    return metadata


def get_metadata(scan_key, session_start_time):
    metadata = get_default_nwbfile_metadata()

    metadata["NWBFile"].update(
        session_start_time=session_start_time,
        subject=get_default_subject(),
        session_id=str(scan_key["session"]),
    )

    get_ophys_metadata(scan_key, metadata)

    return metadata

def add_stimulus(scan_key, nwb):
    timestamps = (nda.FrameTimes() & scan_key).fetch1('frame_times') # timestamps of stimulus images
    movie = (nda.Stimulus & scan_key).fetch1('movie')
    optical_series = OpticalSeries(
        name="visual stimulus",
        distance=np.nan,  # unknown
        field_of_view=[np.nan, np.nan],
        orientation="0 is up",
        data=H5DataIO(movie.transpose(2, 0, 1), compression=True),
        timestamps=H5DataIO(timestamps, compression=True),
        unit="n.a.",
    )

    nwb.add_stimulus(optical_series)

def add_eye_tracking(scan_key, nwb):
    pupil_min_r, pupil_maj_r, pupil_x, pupil_y, timestamps = (
        nda.RawManualPupil() & scan_key
    ).fetch1("pupil_min_r", "pupil_maj_r", "pupil_x", "pupil_y", "pupil_times")

    pupil_min_r = TimeSeries(
        name="pupil_min_r",
        description="minor axis of pupil tracking ellipse",
        data=H5DataIO(pupil_min_r, compression=True),
        timestamps=H5DataIO(timestamps, compression=True),
        unit="unknown",
    )

    pupil_maj_r = TimeSeries(
        name="pupil_maj_r",
        description="jajor axis of pupil tracking ellipse",
        data=H5DataIO(pupil_maj_r, compression=True),
        timestamps=pupil_min_r,
        unit="unknown",
    )

    pupil_tracking = PupilTracking([pupil_min_r, pupil_maj_r])
    nwb.add_acquisition(pupil_tracking)

    pupil_xy = SpatialSeries(
        name = "eye_position",
        description="x,y position of eye",
        data=H5DataIO(np.c_[pupil_x, pupil_y], compression=True),
        timestamps=pupil_min_r,
        unit="unknown",
        reference_frame="unknown",
    )

    eye_position_tracking = EyeTracking(pupil_xy)

    nwb.add_acquisition(eye_position_tracking)

def add_treadmill(scan_key, nwb):
    treadmill_velocity, treadmill_timestamps = (
        nda.RawTreadmill & scan_key
    ).fetch1("treadmill_velocity", "treadmill_timestamps")

    treadmill_velocity = TimeSeries(
        name="treadmill_velocity",
        data=H5DataIO(treadmill_velocity, compression=True),
        timestamps=H5DataIO(treadmill_timestamps, compression=True),
        description="velocity of treadmill",
        unit="unknown",
    )

    nwb.add_acquisition(treadmill_velocity)


def add_trials(scan_key, nwb):

    nwb.add_trial_column("condition_hash", "condition hash")
    nwb.add_trial_column("stimulus_type", "stimulus type")


    stimulus_types, start_times, stop_times, condition_hashes, trial_idxs = (
        nda.Trial & scan_key
    ).fetch("type", "start_frame_time", "end_frame_time", "condition_hash", "trial_idx", order_by="trial_idx")

    for stimulus_type, start_time, stop_time, condition_hash, trial_idx in zip(
        stimulus_types, start_times, stop_times, condition_hashes, trial_idxs
    ):
        nwb.add_trial(
            id=trial_idx,
            start_time=start_time,
            stop_time=stop_time,
            stimulus_type=stimulus_type,
            condition_hash=condition_hash,
        )

def add_plane_segmentation(field_key, nwb):
    ophys = get_module(nwb, "ophys")
    image_segmentation = ophys.data_interfaces['ImageSegmentation']

    imaging_plane = nwb.imaging_planes[f"ImagingPlane{field_key['field']}"]

    ps = image_segmentation.create_plane_segmentation(
        name=f"PlaneSegmentation{field_key['field']}",
        description='output from segmenting my favorite imaging plane',
        imaging_plane=imaging_plane,
    )
    ps.add_column("mask_type", "type of ROI")

    image_height, image_width = (nda.Field & field_key).fetch1("px_height", "px_width")

    mask_pixels, mask_weights, mask_ids, mask_types = (nda.Segmentation * nda.MaskClassification & field_key).fetch(
        "pixels", "weights", "mask_id", "mask_type", order_by="mask_id"
    )

    # Reshape masks
    masks = func.reshape_masks(mask_pixels, mask_weights, image_height, image_width)
    masks = masks.transpose((2,0,1))


    for image_mask, mask_id, mask_type in zip(masks, mask_ids, mask_types):
        ps.add_roi(
            image_mask=image_mask,
            id=mask_id,
            mask_type=mask_type,
        )

    return ps


def add_roi_response_series(field_key, nwb, plane_segmentation):

    frame_times = (nda.FrameTimes & scan_key).fetch1('frame_times')

    data = np.vstack((nda.Fluorescence() & field_key).fetch("trace", order_by="mask_id")).T

    rt_region = plane_segmentation.create_roi_table_region(
        region=list(range(data.shape[1])),
        description=f"all rois in field {field_key['field']}"
    )

    roi_response_series = RoiResponseSeries(
        name=f"RioResponseSeries{field_key['field']}",
        description=f"traces for field {field_key['field']}",
        data=H5DataIO(data, compression=True),
        rois=rt_region,
        timestamps=H5DataIO(frame_times, compression=True),
        unit="n/a",
    )

    fluorescence = Fluorescence()
    fluorescence.add_roi_response_series(roi_response_series)

    ophys = get_module(nwb, "ophys")
    ophys.add(fluorescence)


def add_summary_images(field_key, nwb):
    ophys = get_module(nwb, "ophys")

    correlation, average = (
        nda.SummaryImages & field_key
    ).fetch1("correlation", "average")

    segmentation_images = Images(f"SegmentationImages{field_key['field']}")

    segmentation_images.add_image(GrayscaleImage(name="Correlation", data=correlation))
    segmentation_images.add_image(GrayscaleImage(name="Average", data=average))

    ophys.add(segmentation_images)


def add_ophys(scan_key, nwb, metadata):
    add_image_segmentation(nwbfile=nwb, metadata=metadata)

    all_field_data = (nda.Field & scan_key).fetch(as_dict=True)

    for field_ind, field_key in enumerate(all_field_data):
        add_imaging_plane(nwbfile=nwb, metadata=metadata, imaging_plane_index=field_ind)
        plane_segmentation = add_plane_segmentation(field_key, nwb)
        add_roi_response_series(field_key, nwb, plane_segmentation)
        add_summary_images(field_key, nwb)


def run_conversion(nwbfile_path: str, session_start_time: datetime, scan_key: dict):
    metadata = get_metadata(scan_key=scan_key, session_start_time=session_start_time)

    with make_or_load_nwbfile(
        nwbfile_path=nwbfile_path,
        metadata=metadata,
        overwrite=False,
        verbose=True,
    ) as nwbfile_out:
        add_ophys(scan_key, nwbfile_out, metadata)
        add_eye_tracking(scan_key, nwbfile_out)
        add_treadmill(scan_key, nwbfile_out)
        add_trials(scan_key, nwbfile_out)
        #add_stimulus(scan_key, nwbfile_out)


In [ ]:
scan_key = {'session': 4, 'scan_idx': 10}
session_start_time = datetime(2020, 1, 1, 12, 30, 0, tzinfo=tz.gettz("US/Pacific"))
nwbfile_path = "test.nwb"

run_conversion(nwbfile_path=nwbfile_path, session_start_time=session_start_time, scan_key=scan_key)

In [ ]:
from pynwb import NWBHDF5IO
io = NWBHDF5IO(nwbfile_path, mode="r")
nwbfile = io.read()

In [ ]:
nda.Scan()

In [ ]:
nda.Trial()

In [ ]:
field_data = dict(field=1)

field_key = {**scan_key, **dict(field=field_data["field"])}

nda.Fluorescence() & field_key

In [ ]:
func.reshape_masks??

In [ ]:
image_height, image_width = (nda.Field & field_key).fetch1(
        "px_height", "px_width"
    )

In [ ]:
image_height

In [ ]:
nda.MaskClassification()

In [ ]:
func.get_all_masks??

In [ ]:
scan_key = {'session': 4, 'scan_idx': 7} 

(nda.Field & scan_key).fetch(as_dict=True)

In [ ]:
np.c_[pupil_x, pupil_y]

In [ ]:
pupil_min_r, pupil_maj_r, pupil_x, pupil_y = (
    nda.RawManualPupil() & scan_key
).fetch("pupil_min_r", "pupil_maj_r", "pupil_x", "pupil_y")

In [ ]:
nda.RawManualPupil()

In [ ]:
len(pupil_maj_r[0])

In [ ]:
movie_times = (nda.FrameTimes() & scan_key).fetch1('frame_times') # timestamps of stimulus images
movie_times

In [ ]:
plt.imshow(movie[...,3400])